In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

from IPython.display import clear_output
from datetime import datetime

# = = = = = = = = = = = = = = =

import os
import shutil

if os.path.exists('./file/2.url'):
    shutil.rmtree('./file/2.url')

os.makedirs('./file/2.url')

# = = = = = = = = = = = = = = =

import pandas as pd

# = = = = = = = = = = = = = = =

import requests

import sys
sys.path.append('../00.Tools')
from crawler_configuration import get_header, get_proxy

# = = = = = = = = = = = = = = =

import json

# = = = = = = = = = = = = = = =

from bs4 import BeautifulSoup
from lxml import etree

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/1.item_number'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

for file in list_file:
    
    # = = = = = = = = = = = = = = =

    output_correct, output_error = pd.DataFrame(), pd.DataFrame()

    # = = = = = = = = = = = = = = =

    input_ = pd.read_excel(f'./file/1.item_number/{file}',
                           header=0,
                           dtype=str).fillna('')
    
    input_['No'] = input_['No'].astype(int)

    print(f'总数量：{len(input_)}')
    print()

    # = = = = = = = = = = = = = = =


    work = Queue()
    for a in range(len(input_)):
        work.put_nowait(a)

    # = = = = = = = = = = = = = = =

    def crawler():
        global output_correct, output_error

        while not work.empty():
            a = work.get_nowait()

            # = = = = = = = = = = = = = = =

            crawler_status = 'error'

            # = = = = = = = = = = = = = = =

            try:
                b = 0
                while True:
                    b += 1
                    
                    try:
                        request_url = f'''https://www.ebay.com/itm/{input_.loc[a, 'Item Number']}?_ul=US&_stpos=91710&orig_cvip=true'''
                        
                        resp = requests.get(request_url,
                                            headers=get_header(ua=False),
                                            proxies=get_proxy(),
                                            timeout=(10, 10))

                        if resp.status_code == 200 or resp.status_code == 404:
                            break
                    except KeyboardInterrupt:
                        break
                    except:
                        continue

                # = = = = = = = = = = = = = = =

                if resp.status_code == 404:
                    raise

                # = = = = = = = = = = = = = = =

                dict_ = json.loads(resp.text.split('$MC=(window.$MC||[]).concat(')[1].split(']}]})</script>')[0] + ']}]}')

                # = = = = = = = = = = = = = = =

                if dict_['o']['g']['itemId'].strip() != input_.loc[a, 'Item Number']:
                    raise

                # = = = = = = = = = = = = = = =

                index_model = -1
                for i in range(len(dict_['o']['w'])):
                    if 'model' in dict_['o']['w'][i][2]:
                        index_model = i
                        break
                
                # = = = = = = = = = = = = = = =

                if index_model == -1:
                    raise
                
                # = = = = = = = = = = = = = = =

                for data in dict_['o']['w'][index_model][2]['model']['modules']['MARS']['metaData']:
                    if 'hreflang="x-default"' in data:
                        soup = BeautifulSoup(data, 'lxml')
                        html = etree.HTML(str(soup))
                
                        url = html.xpath('//link/@href')[0]
                
                        if url.startswith('https://www.ebay.com/'):
                            url = f'{url}?_ul=US&_stpos=91710&orig_cvip=true'
                        elif url.startswith('https://www.ebay.de/'):
                            url = f'{url}?_ul=DE&_stpos=10115&orig_cvip=true'
                        elif url.startswith('https://www.ebay.co.uk/'):
                            url = f'{url}?_ul=GB&_stpos=BS81QU&orig_cvip=true'
                        elif url.startswith('https://www.ebay.com.au/'):
                            url = f'{url}?_ul=AU&_stpos=2019&orig_cvip=true'
                        elif url.startswith('https://www.ebay.ca/'):
                            url = f'{url}?_ul=CA&_stpos=M5S2E8&orig_cvip=true'
                        else:
                            raise

                # = = = = = = = = = = = = = = =

                df_temp = pd.DataFrame([{'No': input_.loc[a, 'No'],
                                         'Item Number': input_.loc[a, 'Item Number'],
                                         'Url': url}])

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

                # = = = = = = = = = = = = = = =

                crawler_status = 'ok'
            except:
                df_temp = pd.DataFrame([input_.iloc[a]]).reset_index(drop=True)
                df_temp.loc[0, 'Request_Url'] = request_url
                
                output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

            # = = = = = = = = = = = = = = =

            if list_file.index(file) % 3 == 0 and list_file.index(file) != 0 and work.qsize() == 0 or work.qsize() % 1_000 == 0 and work.qsize() != 0:
                clear_output()

            print(f'''[状态：{crawler_status}，尝试次数：{b}] - {(list_file.index(file) + 1) / len(list_file) * 100:.2f}% - {file.removesuffix('.xlsx')} > {input_.loc[a, 'No']}.{request_url}\n[剩余数量：{work.qsize()}] - [当前时间：{datetime.now().strftime('%H:%M:%S')}]\n''')

    # = = = = = = = = = = = = = = =

    list_task = []
    for _ in range(5):
        task = gevent.spawn(crawler)
        list_task.append(task)
    gevent.joinall(list_task)

    print('输出ing...')
    print()
    if not output_correct.empty:
        output_correct = output_correct.sort_values(by=['No'],
                                                    ascending=[True],
                                                    ignore_index=True)
        output_correct.to_excel(f'''./file/2.url/{file.removesuffix('.xlsx')}-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx''', index=False)
    if not output_error.empty:
        output_error = output_error.sort_values(by=['No'],
                                                ascending=[True],
                                                ignore_index=True)
        output_error.to_excel(f'''./file/{file.removesuffix('.xlsx')}-url_error.xlsx''', index=False)
        print('爬虫存在error')
        print()

# = = = = = = = = = = = = = = =

list_file = sorted(list(os.walk('./file/2.url'))[0][2])
while '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
while '0.null.txt' in list_file:
    list_file.remove('0.null.txt')

# = = = = = = = = = = = = = = =

from tqdm import tqdm
import re

for file in tqdm(list_file, desc='Progress', ncols=77):
    os.rename(f'./file/2.url/{file}',
              f'''./file/2.url/{re.sub(r'-[0-9]{8}_[0-9]{6}.xlsx$', '.xlsx', file)}''')

print('Done ~')

总数量：10

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 3.https://www.ebay.com/itm/235653299500?_ul=US&_stpos=91710&orig_cvip=true
[剩余数量：5] - [当前时间：11:23:03]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 1.https://www.ebay.com/itm/253139036562?_ul=US&_stpos=91710&orig_cvip=true
[剩余数量：4] - [当前时间：11:23:04]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 7.https://www.ebay.com/itm/256421824484?_ul=US&_stpos=91710&orig_cvip=true
[剩余数量：3] - [当前时间：11:23:05]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 6.https://www.ebay.com/itm/332388018897?_ul=US&_stpos=91710&orig_cvip=true
[剩余数量：2] - [当前时间：11:23:06]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 8.https://www.ebay.com/itm/334041367847?_ul=US&_stpos=91710&orig_cvip=true
[剩余数量：1] - [当前时间：11:23:07]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 9.https://www.ebay.com/itm/256102649999?_ul=US&_stpos=91710&orig_cvip=true
[剩余数量：0] - [当前时间：11:23:07]

[状态：ok，尝试次数：1] - 50.00% - crawler_1 > 5.https://www.ebay.com/itm/233621258535?_ul=US&_stpos=91710&orig_cvip=true
[剩余数量：0] - [当前时间：11:23:08]

[状态：o

Progress: 100%|██████████████████████████████| 2/2 [00:00<00:00, 1959.04it/s]

Done ~
